## Project Context: Intelligent Invoice Parsing using Databricks AI

In many real-world teams, invoices and receipts arrive as PDFs or scanned images, each following a different layout and vendor format.
While humans can visually understand these documents easily, extracting structured information from them at scale is slow, manual, and error-prone.

The goal of this project is to build an **intelligent document parsing pipeline** that can:
- Accept raw invoices in PDF or image format
- Automatically understand document layout and content
- Extract key business fields such as invoice number, subtotal, tax, and total
- Convert unstructured documents into structured, analytics-ready tables

This notebook demonstrates how Databricks’ `ai_parse_document` capability can be combined with Spark-based data engineering to achieve this in a scalable and production-oriented way.


In [0]:
#location for raw invoice PDFs.
base_path = "/Volumes/workspace/gs_invoices/raw_invoices"

In [0]:
#visually inspect PDF names and sizes
files = dbutils.fs.ls(base_path)
display(files)


path,name,size,modificationTime
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/HandwrittenInvoice1.jpg,HandwrittenInvoice1.jpg,666505,1769047214000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,Invoice_BioHue_Chemicals_102.pdf,2687,1769047188000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,Invoice_BioHue_Chemicals_13.pdf,2695,1769047188000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_30.pdf,Invoice_BioHue_Chemicals_30.pdf,2689,1769047188000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_35.pdf,Invoice_BioHue_Chemicals_35.pdf,2680,1769047188000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_36.pdf,Invoice_BioHue_Chemicals_36.pdf,2688,1769047188000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_37.pdf,Invoice_BioHue_Chemicals_37.pdf,2699,1769047188000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_38.pdf,Invoice_BioHue_Chemicals_38.pdf,2689,1769047188000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_43.pdf,Invoice_BioHue_Chemicals_43.pdf,2704,1769047188000
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_7.pdf,Invoice_BioHue_Chemicals_7.pdf,2682,1769047188000


## Reading Raw Documents (PDFs)

Invoices and receipts are not plain text files they are binary documents.
To preserve their original structure and visual layout, we read them as binary data instead of attempting text extraction at this stage.

At this point, we are not interpreting or extracting meaning.
We are simply loading the raw documents into the system so they can later be processed by an AI-powered document parser.


In [0]:
## Ingest raw invoice PDFs as binary files.
## raw binary content (required for ai_parse_document),

raw_df = (
  spark.read.format("binaryFile")
  .option("pathGlobFilter", "*.pdf")
  .load(base_path)
  .select("path", "content", "length", "modificationTime")
)

display(raw_df)

path,content,length,modificationTime
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf,JVBERi0xLjQKJZOMi54gUmVwb3J0TGFiIEdlbmVyYXRlZCBQREYgZG9jdW1lbnQgaHR0cDovL3d3dy5yZXBvcnRsYWIuY29tCjEgMCBvYmoKPDwKL0YxIDIgMCBSIC9GMiAzIDAgUiAvRjMgNCAwIFIKPj4KZW5kb2JqCjIgMCBvYmoKPDwKL0Jhc2VGb250IC9IZWx2ZXRpY2EgL0VuY29kaW5nIC9XaW5BbnNpRW5jb2RpbmcgL05hbWUgL0YxIC9TdWJ0eXBlIC9UeXBlMSAvVHlwZSAvRm9udAo+PgplbmRvYmoKMyAwIG9iago8PAovQmFzZUZvbnQgL0hlbHZldGljYS1Cb2xkIC9FbmNvZGluZyAvV2luQW5zaUVuY29kaW5nIC9OYW1lIC9GMiAvU3VidHlwZSAvVHlwZTEgL1R5cGUgL0ZvbnQKPj4KZW5kb2JqCjQgMCBvYmoKPDwKL0Jhc2VGb250IC9IZWx2ZXRpY2EtQm9sZE9ibGlxdWUgL0VuY29kaW5nIC9XaW5BbnNpRW5jb2RpbmcgL05hbWUgL0YzIC9TdWJ0eXBlIC9UeXBlMSAvVHlwZSAvRm9udAo+PgplbmRvYmoKNSAwIG9iago8PAovQ29udGVudHMgOSAwIFIgL01lZGlhQm94IFsgMCAwIDYxMiA3OTIgXSAvUGFyZW50IDggMCBSIC9SZXNvdXJjZXMgPDwKL0ZvbnQgMSAwIFIgL1Byb2NTZXQgWyAvUERGIC9UZXh0IC9JbWFnZUIgL0ltYWdlQyAvSW1hZ2VJIF0KPj4gL1JvdGF0ZSAwIC9UcmFucyA8PAoKPj4gCiAgL1R5cGUgL1BhZ2UKPj4KZW5kb2JqCjYgMCBvYmoKPDwKL1BhZ2VNb2RlIC9Vc2VOb25lIC9QYWdlcyA4IDAgUiAvVHlwZSAvQ2F0YWxvZwo+PgplbmRvYmoKNyAwIG9iago8PAovQXV0aG9yIChcKGFub255bW91c1wpKSAvQ3JlYXRpb25EYXRlIChEOjIwMjUxMDAzMjI0NDQxKzAwJzAwJykgL0NyZWF0b3IgKFwodW5zcGVjaWZpZWRcKSkgL0tleXdvcmRzICgpIC9Nb2REYXRlIChEOjIwMjUxMDAzMjI0NDQxKzAwJzAwJykgL1Byb2R1Y2VyIChSZXBvcnRMYWIgUERGIExpYnJhcnkgLSB3d3cucmVwb3J0bGFiLmNvbSkgCiAgL1N1YmplY3QgKFwodW5zcGVjaWZpZWRcKSkgL1RpdGxlIChcKGFub255bW91c1wpKSAvVHJhcHBlZCAvRmFsc2UKPj4KZW5kb2JqCjggMCBvYmoKPDwKL0NvdW50IDEgL0tpZHMgWyA1IDAgUiBdIC9UeXBlIC9QYWdlcwo+PgplbmRvYmoKOSAwIG9iago8PAovRmlsdGVyIFsgL0FTQ0lJODVEZWNvZGUgL0ZsYXRlRGVjb2RlIF0gL0xlbmd0aCAxMTM1Cj4+CnN0cmVhbQpHYXQ9Kj4+czlLJSI/TytkKjBWIkpQUDQyPSdwMjU1VWJfZiM/IyJsSUdlXSYvO107UDljXSM8cjs0V0MtUj9NZEttNydSakEyRERQdV8mcSI+PHM3VUQyVztLUlowYFRGQVtCVFlfOVBJWU5jYnBUbUEqViEhQEwwRWUvMDwzZ28mNlQpR2Q2Q2VmbihmN3RYQEJBT18hSEExaiw3bVcva2omJDFjLjJYLzBuX2I2cXVGWEpZQSE+Mz9NYydUW2tyQFlRKGxHNkFmXF4sOC80bl1hTypGZWBwa1YxPWc8WSVDOV49XEUsTTIqLnVoUDtCPXVLJEtTMyxOOkpKOVBjUlJCRyhuODAvPCZQYzRjMFlOIS1FIUNdM0kvNlluX0BbTyJSXz5IUGtXNS5qRSk0RThpbD9JTm8lTCVXSlpsUT1LNGJCczQqNjdlQkApRS1MRVdaUlpdKXQ1PW0nVkdNdClKJmRaYFJRVD08RSxJWWQjbFlKPnM+OjZoI2hiSGxgIzpyJVVqbWVRJkEvRlxfbF1LSmFGP1k2SFJvYk1ARWQhZlRNUSUyXkBEKjA8K1EmMjgvcE9icWszKzpBMTMnLEtIQmlzMnEvMEFaY3I8M2hZSjFRLStrMT91XGZmKjltbW4yKUs9KzApX1c2ZGtDYVgpLC8/IilvZVo0b2xkU2VMOkA7VjZYS2NAKDhcSGE4LEgrTkA4W0hNQzNAIUNFbXJOKjtITz0ua09sa2A2RXFbTUFbOU4wPDAtLEpxXygtOUVIMnE/MVBxLTJjSjA7Q1g5J1g+Y0tVJE5ab0FZbFtWMCVTOTQwIjhYLi8xdD4jQGxnKlFBW2pWaTFjOz44Qk1HNSo5N0QiRG9LOTw3UTVZS15EWVBySUZaLjM2P1skcjlYNj9oLUBFS1ElcGAmVWQ5JmNILjBIUCUsSTg7bldbIkthOWopSnVGRlpaZls/NyJndHAyXDRbRCUzVTpqWU4xQCZWUC5UY1coPUlmX2dXKSlDR05LTzBlYEQoPmhdNl9mZUM2PjctakhlbnA/MTthcmVsVXBxN14kUGhZJD5McjdtcSNBYF8sYl1jXmlVRCw+TTRgYCs/QT0rbWU1SXBqbDZWIVEscFRnUmxxKUBDcD5aTkQvMUUqZVFdMkREU3NBOG1RMitKNGhiZHNtTjlwdF9bKTJDRSdhZWk0amcsXlxsdCxVSUw6dGZfYGc0ZS8+SFk/KEJzUlNBUXBOKU5ic280ZWk+WlUqOzssI21fS1AlXzRDMFN1Q3Q7KDQtJVIjQiM1PFEmXzIsRyMiR1EsQkdvZ3RkaHRQMG5OZD9gIzxUbUJIM2gzWz4iajopaCVHOFYyQSQ4OF9HYExuUFs6V1dkWyRRT11rLlA3PUxOQV9EX1YyKkxfO2wwUE5FUDg4Nk0mJSJvQW4kS0dGVWU5TSxtN1lXSSEndVc1aGVFNVhlUSc8OGZMVnMrV2JuRzY5ZlQqQW5ASFJWTTM0bEhnQDJyPD1WI1pzan4+ZW5kc3RyZWFtCmVuZG9iagp4cmVmCjAgMTAKMDAwMDAwMDAwMCA2NTUzNSBmIAowMDAwMDAwMDczIDAwMDAwIG4gCjAwMDAwMDAxMjQgMDAwMDAgbiAKMDAwMDAwMDIzMSAwMDAwMCBuIAowMDAwMDAwMzQzIDAwMDAwIG4gCjAwMDAwMDA0NjIgMDAwMDAgbiAKMDAwMDAwMDY1NSAwMDAwMCBuIAowMDAwMDAwNzIzIDAwMDAwIG4gCjAwMDAwMDEwMDYgMDAwMDAgbiAKMDAwMDAwMTA2NSAwMDAwMCBuIAp0cmFpbGVyCjw8Ci9JRCAKWzwwYmUwNzQ4MWYxMTBiM2FiZjg1NzliN2Y5MzE4NWNkMj48MGJlMDc0ODFmMTEwYjNhYmY4NTc5YjdmOTMxODVjZDI+XQolIFJlcG9ydExhYiBnZW5lcmF0ZWQgUERGIGRvY3VtZW50IC0tIGRpZ2VzdCAoaHR0cDovL3d3dy5yZXBvcnRsYWIuY29tKQoKL0luZm8gNyAwIFIKL1Jvb3QgNiAwIFIKL1NpemUgMTAKPj4Kc3RhcnR4cmVmCjIyOTEKJSVFT0YK,2718,2026-01-22T01:59:49.000Z
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf,JVBERi0xLjQKJZOMi54gUmVwb3J0TGFiIEdlbmVyYXRlZCBQREYgZG9jdW1lbnQgaHR0cDovL3d3dy5yZXBvcnRsYWIuY29tCjEgMCBvYmoKPDwKL0YxIDIgMCBSIC9GMiAzIDAgUi

## Intelligent Document Parsing using ai_parse_document

This is the core step of the project.

The `ai_parse_document` function performs more than traditional OCR:
- It reads text from scanned or digital documents
- It understands document layout (pages, blocks, tables)
- It produces a structured representation of the document content

Instead of returning plain text, this step outputs a rich, nested structure describing **what the document contains and how it is organized**.

This structured representation becomes the foundation for all downstream extraction and business logic.


In [0]:

# Parse each raw PDF using Databricks ai_parse_document.
# converting raw binary content into a structured document representation
#The output is stored as a VARIANT column for downstream extraction and validation.

from pyspark.sql.functions import expr

parsed_df = raw_df.select(
    "path",
    expr("ai_parse_document(content) as parsed_document")
)


path
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_99.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_9.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_PureTint_Suppliers_101.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_73.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainPigments_Ltd_64.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_100.pdf
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_NatureChem_Products_25.pdf


## Extracting Document Elements

The parsed document output contains a nested collection of elements, such as:
- Text blocks
- Table cells
- Headers and footers
- Page-level metadata

To make this information usable, we extract the document elements field and prepare it for flattening.


In [0]:
## ai_parse_document returns a nested VARIANT structure, so we first extract the elements array as a JSON string.
# This prepares the data for explicit schema definition and row-level 

elements_json_df = parsed_df.select(
    "path",
    expr("cast(variant_get(parsed_document, '$.document.elements') as string) as elements_json")
)

display(elements_json_df)



path elements_json dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf [{"bbox":[{"coord":[447,133,576,166],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[130,211,416,293],"page_id":0}],"content":"EcoPigments Supply Co.\n45 Harmony Lane, Portland, OR, USA\nPhone: (555) 191-9410\nEmail: sales@ecopigments.com","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[130,329,187,346],"page_id":0}],"content":"Bill To:","description":null,"id":2,"type":"section_header"},{"bbox":[{"coord":[130,357,265,413],"page_id":0}],"content":"GreenSheen\n123 Eco Drive\nDenver, CO, USA","description":null,"id":3,"type":"text"},{"bbox":[{"coord":[186,435,526,567],"page_id":0}],"content":" Invoice Number: INV-30016 Invoice Date: 2025-10-19 Due Date: 2025-11-02 Payment Terms: Net 14 ","description":null,"id":4,"type":"table"},{"bbox":[{"coord":[83,594,935,682],"page_id":0}],"content":" Item Code Description Quantity Unit Price Amount PIG-777 Eco-Friendly Pigment Pack 9 $75.00 $675.00 ORG-201 Organic Titanium Dioxide (25kg) 12 $120.00 $1440.00 ","description":null,"id":5,"type":"table"},{"bbox":[{"coord":[91,709,929,822],"page_id":0}],"content":" Subtotal: $2115.00 Shipping: $83.00 Tax (5.5%): $116.33 Total Due: $2314.32 ","description":null,"id":6,"type":"table"},{"bbox":[{"coord":[131,844,616,884],"page_id":0}],"content":"Thank you for choosing EcoPigments Supply Co..\nPlease make payment to account #78XXXX019, Bank of America.","description":null,"id":7,"type":"text"}] dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf [{"bbox":[{"coord":[447,133,576,166],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[130,211,428,293],"page_id":0}],"content":"GreenCore Additives\n67 Pine Grove, Salt Lake City, UT, USA\nPhone: (555) 498-6352\nEmail: orders@greencore.com","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[130,329,187,346],"page_id":0}],"content":"Bill To:","description":null,"id":2,"type":"section_header"},{"bbox":[{"coord":[130,357,265,413],"page_id":0}],"content":"GreenSheen\n123 Eco Drive\nDenver, CO, USA","description":null,"id":3,"type":"text"},{"bbox":[{"coord":[186,435,526,567],"page_id":0}],"content":" Invoice Number: INV-30089 Invoice Date: 2025-12-31 Due Date: 2026-01-14 Payment Terms: Net 14 ","description":null,"id":4,"type":"table"},{"bbox":[{"coord":[82,591,938,687],"page_id":0}],"content":" Item Code Description Quantity Unit Price Amount PIG-777 Eco-Friendly Pigment Pack 2 $75.00 $150.00 ORG-201 Organic Titanium Dioxide (25kg) 11 $120.00 $1320.00 ","description":null,"id":5,"type":"table"},{"bbox":[{"coord":[91,698,929,824],"page_id":0}],"content":" Subtotal: $1470.00 Shipping: $83.00 Tax (5.5%): $80.85 Total Due: $1633.85 ","description":null,"id":6,"type":"table"},{"bbox":[{"coord":[130,844,616,884],"page_id":0}],"content":"Thank you for choosing GreenCore Additives.\nPlease make payment to account #78XXXX019, Bank of America.","description":null,"id":7,"type":"text"}] dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf [{"bbox":[{"coord":[447,133,576,166],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[130,211,283,230],"page_id":0}],"content":"SustainSolvents Inc.","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[130,230,377,250],"page_id":0}],"content":"12 River Bend, Chicago, IL, USA","description":null,"id":2,"type":"text"},{"bbox":[{"coord":[130,250,306,271],"page_id":0}],"content":"Phone: (555) 805-8887","description":null,"id":3,"type":"text"},{"bbox":[{"coord":[130,271,405,290],"page_id":0}],"content":"Email: contact@sustainsolvents.com","description":null,"id":4,"type":"text"},{"bbox":[{"coord":[130,329,190,345],"page_id":0}],"content":"Bill To:","description":null,"id":5,"type":"text"},{"bbox":[{"coord":[130,357,229,373],"page_id":0}]

## Defining a Structured Schema for Document Elements

AI parsing outputs complex nested data.
Instead of relying on schema inference, we explicitly define the schema for document elements.


In [0]:
# STEP 1: Define the structure of a single document element produced by ai_parse_document.
# Each element represents a small piece of the document such as:
# - a line of text
# - a table cell
# - a header or footer
# along with its page number and position (bounding box) in the document.

# STEP 2: Convert the raw JSON string of document elements into a typed Spark structure
# using the schema defined above.


from pyspark.sql.functions import col, from_json, explode

elements_schema = """
array<struct<
  id: int,
  type: string,
  content: string,
  page_id: int,
  bbox: struct<coord: array<int>, page_id: int>,
  description: string
>>
"""

# Explode the elements array so that:
# - each row corresponds to ONE document element
# - a single PDF becomes many rows (one per element)

# After this step, we can easily filter, search, and reason over individual
# parts of the invoice instead of treating the document as a single blob.

elements_df = (
    elements_json_df
    .select("path", explode(from_json(col("elements_json"), elements_schema)).alias("e"))
    .select(
        "path",
        col("e.id").alias("element_id"),
        col("e.type").alias("element_type"),
        col("e.content").alias("content"),
        col("e.page_id").alias("page_id"),
        col("e.bbox.coord").alias("bbox_coord"),
        col("e.description").alias("description")
    )
)

display(elements_df)


path element_id element_type content page_id bbox_coord description dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf 0 section_header INVOICE null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf 1 text EcoPigments Supply Co.
45 Harmony Lane, Portland, OR, USA
Phone: (555) 191-9410
Email: sales@ecopigments.com null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf 2 section_header Bill To: null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf 3 text GreenSheen
123 Eco Drive
Denver, CO, USA null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf 4 table Invoice Number: INV-30016 Invoice Date: 2025-10-19 Due Date: 2025-11-02 Payment Terms: Net 14 null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf 5 table Item Code Description Quantity Unit Price Amount PIG-777 Eco-Friendly Pigment Pack 9 $75.00 $675.00 ORG-201 Organic Titanium Dioxide (25kg) 12 $120.00 $1440.00 null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf 6 table Subtotal: $2115.00 Shipping: $83.00 Tax (5.5%): $116.33 Total Due: $2314.32 null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_16.pdf 7 text Thank you for choosing EcoPigments Supply Co..
Please make payment to account #78XXXX019, Bank of America. null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf 0 section_header INVOICE null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf 1 text GreenCore Additives
67 Pine Grove, Salt Lake City, UT, USA
Phone: (555) 498-6352
Email: orders@greencore.com null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf 2 section_header Bill To: null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf 3 text GreenSheen
123 Eco Drive
Denver, CO, USA null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf 4 table Invoice Number: INV-30089 Invoice Date: 2025-12-31 Due Date: 2026-01-14 Payment Terms: Net 14 null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf 5 table Item Code Description Quantity Unit Price Amount PIG-777 Eco-Friendly Pigment Pack 2 $75.00 $150.00 ORG-201 Organic Titanium Dioxide (25kg) 11 $120.00 $1320.00 null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf 6 table Subtotal: $1470.00 Shipping: $83.00 Tax (5.5%): $80.85 Total Due: $1633.85 null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_89.pdf 7 text Thank you for choosing GreenCore Additives.
Please make payment to account #78XXXX019, Bank of America. null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf 0 section_header INVOICE null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf 1 text SustainSolvents Inc. null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf 2 text 12 River Bend, Chicago, IL, USA null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf 3 text Phone: (555) 805-8887 null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf 4 text Email: contact@sustainsolvents.com null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf 5 text Bill To: null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf 6 text GreenSheen null null null dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.

In [0]:
# Identify candidate document elements that may contain invoice-level monetary fields.
# We perform a case-insensitive keyword scan over parsed document content to surface elements related to totals, taxes, shipping, and balances.

from pyspark.sql.functions import lower

keywords = ["total", "subtotal", "tax", "gst", "vat", "shipping", "freight", "discount", "amount due", "balance due"]

invoice_amount_candidates = (
    elements_df
    .withColumn("content_l", lower(col("content")))
    .where(" OR ".join([f"content_l LIKE '%{k}%'" for k in keywords]))
    .select("path", "page_id", "element_type", "content", "bbox_coord")
    .orderBy("path", "page_id", "element_id")
)

display(invoice_amount_candidates)



path,page_id,element_type,content,bbox_coord
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,null,text,Subtotal: $480.00,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,null,text,Shipping: $110.00,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,null,text,Tax (5.5%): $28.40,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,null,text,Total Due: $616.40,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,null,text,Subtotal: $555.00,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,null,text,Shipping: $63.00,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,null,text,Tax (5.5%): $30.52,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,null,text,Total Due: $640.52,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_30.pdf,null,text,Subtotal: $33600.00,null
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_30.pdf,null,text,Shipping: $100.00,null


## Extracting Monetary Fields from Candidate Elements

After identifying candidate elements, we clean and normalize the extracted text to remove HTML artifacts, extra whitespace, and casing inconsistencies.

We then extract invoice-level monetary fields (subtotal, shipping, tax, total) using label-based patterns.
This step is intentionally tolerant and works even when multiple fields appear in the same table row.

Finally, we reshape the extracted values into a canonical `(field, amount)` format, making the data easier to validate, aggregate, and analyze downstream.


In [0]:
from pyspark.sql import functions as F

# 1) Clean HTML -> plain text : Clean the extracted document text, The parsed document content may contain HTML tags, extra spaces, and mixed casing.
pairs_src = (
    invoice_amount_candidates
    .withColumn("plain", F.regexp_replace(F.col("content"), r"<[^>]+>", " "))
    .withColumn("plain", F.regexp_replace(F.col("plain"), r"\s+", " "))
    .withColumn("plain_l", F.lower(F.col("plain")))
)

# 2) Extract each field from the same row (works even if all fields are in one table row)
invoice_amounts = (
    pairs_src
    .select(
        "path",
        F.regexp_extract("plain", r"Subtotal:\s*\$?\s*(\d[\d,]*\.?\d{0,2})", 1).alias("subtotal"),
        F.regexp_extract("plain", r"Shipping:\s*\$?\s*(\d[\d,]*\.?\d{0,2})", 1).alias("shipping"),
        F.regexp_extract("plain", r"Tax[^:]*:\s*\$?\s*(\d[\d,]*\.?\d{0,2})", 1).alias("tax"),
        F.regexp_extract("plain", r"(Total Due:|Amount Due:|Balance Due:)\s*\$?\s*(\d[\d,]*\.?\d{0,2})", 2).alias("total")
    )
    # 3) Convert wide columns into a long (field, amount) format
    # We reshape the data so that:
            # - each row represents ONE extracted amount
            # - columns become: (invoice path, field name, numeric amount)
    .selectExpr(
        "path",
        "stack(4, 'subtotal', subtotal, 'shipping', shipping, 'tax', tax, 'total', total) as (field, amount_str)"
    )
    .withColumn("amount", F.expr("try_cast(regexp_replace(amount_str, '[, ]', '') as double)")) #Clean extracted amounts
    .where(F.col("amount").isNotNull())
    .select("path", "field", "amount")
)

display(invoice_amounts)


path,field,amount
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,subtotal,480.0
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,shipping,110.0
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,tax,26.4
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_102.pdf,total,616.4
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,subtotal,555.0
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,shipping,63.0
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,tax,30.52
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_13.pdf,total,640.52
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_30.pdf,subtotal,33600.0
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_30.pdf,shipping,100.0


## Aggregating Extracted Fields to Invoice Level

Earlier steps produced multiple rows per invoice, as each document was decomposed into individual elements.
Here, we reconstruct a **one-row-per-invoice** view.

We first create a complete list of invoice paths, then pivot extracted `(field, amount)` values back into columns
(`subtotal`, `shipping`, `tax`, `total`).

A left join ensures that invoices with missing or partially extracted fields are still included,
making gaps explicit rather than silently dropping data.


In [0]:
## Create a one-row-per-invoice list -Earlier, each invoice was split into many rows (one per document element).
# Here, we deduplicate paths so every invoice is tracked in the final output.

invoice_files = (
    elements_df
    .select("path")
    .distinct()
)


In [0]:

# Reshape extracted amounts back to one row per invoice.
# We pivot (field -> columns) to create subtotal, shipping, tax, and total columns,
# then left join with the full invoice list so invoices with missing values are still included for validation and reporting.


from pyspark.sql import functions as F

all_paths = invoice_files.select("path").distinct()

pivot_financials = (
    invoice_amounts
    .groupBy("path")
    .pivot("field", ["subtotal", "shipping", "tax", "total"])
    .agg(F.max("amount"))
)

invoice_financials = (
    all_paths
    .join(pivot_financials, on="path", how="left")
)

display(invoice_financials)


path,subtotal,shipping,tax,total
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoBase_Minerals_5.pdf,1890.0,63.0,103.95,2056.95
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_43.pdf,2220.0,61.0,122.1,2403.1
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf,1825.0,80.0,100.38,2005.38
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_PureTint_Suppliers_101.pdf,1470.0,53.0,80.85,1603.85
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_99.pdf,1755.0,55.0,96.53,1906.53
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_PureTint_Suppliers_81.pdf,350.0,148.0,19.25,517.25
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_NatureChem_Products_25.pdf,1770.0,138.0,97.35,2005.35
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_73.pdf,1605.0,55.0,88.28,1748.28
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_100.pdf,1395.0,98.0,76.72,1569.72
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainPigments_Ltd_64.pdf,1710.0,84.0,94.05,1888.05


## Validating Extracted Totals and Flagging Issues

Extraction alone is not enough, we also need trust signals.

Here we compute a `calculated_total = subtotal + shipping + tax` (treating missing values as 0),
compare it with the extracted `total`, and record the absolute difference.

We then assign a simple `status` label to make issues explicit (missing fields or total mismatch),
so the final output is not just extracted data, but **validated, debuggable invoice-level results**.


In [0]:
invoice_gold = (
    invoice_financials
    .withColumn(
        "calculated_total",
        F.coalesce(F.col("subtotal"), F.lit(0.0)) +
        F.coalesce(F.col("shipping"), F.lit(0.0)) +
        F.coalesce(F.col("tax"), F.lit(0.0))
    )
    .withColumn("diff", F.abs(F.col("total") - F.col("calculated_total")))
    .withColumn("is_consistent", F.col("diff") <= F.lit(0.05))
    .withColumn(
        "status",
        F.when(F.col("total").isNull(), "MISSING_TOTAL")
         .when(F.col("subtotal").isNull(), "MISSING_SUBTOTAL")
         .when(F.col("shipping").isNull(), "MISSING_SHIPPING")
         .when(F.col("tax").isNull(), "MISSING_TAX")
         .when(~F.col("is_consistent"), "TOTAL_MISMATCH")
         .otherwise("OK")
    )
)


In [0]:
display(invoice_gold)

path,subtotal,shipping,tax,total,calculated_total,diff,is_consistent,status
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoBase_Minerals_5.pdf,1890.0,63.0,103.95,2056.95,2056.95,0.0,true,OK
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_BioHue_Chemicals_43.pdf,2220.0,61.0,122.1,2403.1,2403.1,0.0,true,OK
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_71.pdf,1835.0,80.0,100.38,2005.38,2015.38,10.0,false,TOTAL_MISMATCH
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_PureTint_Suppliers_101.pdf,1470.0,53.0,80.85,1603.85,1603.85,0.0,true,OK
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_EcoPigments_Supply_Co_99.pdf,1755.0,55.0,96.53,1906.53,1906.53,0.0,true,OK
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_PureTint_Suppliers_81.pdf,350.0,148.0,19.25,517.25,517.25,0.0,true,OK
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_NatureChem_Products_25.pdf,1770.0,138.0,97.35,2005.35,2005.35,0.0,true,OK
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_GreenCore_Additives_73.pdf,1605.0,55.0,88.28,1748.28,1748.28,0.0,true,OK
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainSolvents_Inc_100.pdf,13385.0,98.0,76.72,1569.72,13559.72,11990.0,false,TOTAL_MISMATCH
dbfs:/Volumes/workspace/gs_invoices/raw_invoices/Invoice_SustainPigments_Ltd_64.pdf,1710.0,84.0,94.05,1888.05,1888.05,0.0,true,OK


## the Final Gold Table

After extraction, aggregation, and validation, we persist the final invoice-level dataset as a managed table.

This `invoice_gold` table represents the **clean, validated, one-row-per-invoice output** of the pipeline
and can now be used directly for analytics, reporting, or downstream AI applications.


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gs_invoices")

invoice_gold.write.mode("overwrite").saveAsTable("gs_invoices.invoice_gold")
